In [56]:
import pandas as pd

In [73]:
population_data_file_path = '../data/raw/Population by borough 1939 to 2039.xlsx'
population_data = pd.read_excel(
    population_data_file_path,
    sheet_name='Population',
    skiprows=3,
    usecols='B:P',
    na_values=['n/a', 'N/A', 'N/A', 'n/a'],
    engine='openpyxl'
)

population_data

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,NaN,mid-year estimate,mid-year estimate,Census,NaN,Projection,Projection,Projection,Projection,NaN,Growth\n1939-2015,NaN,NaN,Growth\n2014-2015,NaN
1,Area Name,1939,1988,2011,NaN,2015,2021,2031,2039,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,total,%,NaN,total,%
3,City of London,9,6.2,7.375,NaN,8.101776,9.559778,10.843221,11.623187,NaN,-0.898224,-0.099803,NaN,0.211497,0.026157
4,Barking and Dagenham,184,156.6,185.911,NaN,205.434874,224.332972,250.46011,271.193003,NaN,21.434874,0.116494,NaN,3.949193,0.019904
5,Barnet,296,289.5,356.386,NaN,393.215093,415.066095,448.240994,469.008723,NaN,97.215093,0.328429,NaN,5.96088,0.015861
6,Bexley,179,217.7,231.997,NaN,237.012319,243.193377,253.74122,258.484219,NaN,58.012319,0.324091,NaN,2.239583,0.009392
7,Brent,310,246.4,311.215,NaN,326.338839,348.633248,374.68888,381.645253,NaN,16.338839,0.052706,NaN,2.624403,0.008207
8,Bromley,237,295.2,309.392,NaN,320.083395,328.780563,342.514284,350.868707,NaN,83.083395,0.350563,NaN,3.484156,0.010872
9,Camden,301,179,220.338,NaN,230.283248,241.602601,257.415289,266.825465,NaN,-70.716752,-0.234939,NaN,3.069207,0.01326


In [74]:
# Combine the first two rows into a single header, ignore the growth rate columns for now
population_data.columns = [
    f"{col1} {col2}" if pd.notna(col1) and pd.notna(col2) 
    else col2
    for col1, col2 in zip(population_data.iloc[0], population_data.iloc[1])
]

# Drop the first three rows
population_data = population_data.drop(index=[0, 1, 2])

# Drop all columns with all NaN values
population_data = population_data.dropna(axis=1, how='all')

# Drop all rows with all NaN values
population_data = population_data.dropna(axis=0, how='all')

# Drop the last 6 rows
population_data = population_data.drop(index=population_data.index[-6:])

# Reset the index
population_data = population_data.reset_index(drop=True)

# Rename columns: replace spaces with underscores and lowercase them
population_data.columns = [
    col.strip().lower().replace(" ", "_").replace("-", "_") if isinstance(col, str) else col
    for col in population_data.columns
]

# Multiply the values in the population columns by 1000
population_columns = [
    'mid_year_estimate_1939', 
    'mid_year_estimate_1988',
    'census_2011', 
    'projection_2015', 
    'projection_2021', 
    'projection_2031',
    'projection_2039'
]
for col in population_columns:
    population_data[col] = population_data[col].astype(str).str.replace(' ', '').str.replace(',', '').astype(float) * 1000
    population_data[col] = population_data[col].astype(int)

# Drop the columns with NaN as the name
population_data = population_data.loc[:, ~population_data.columns.isna()]

In [75]:
print(population_data.columns)

Index(['area_name', 'mid_year_estimate_1939', 'mid_year_estimate_1988',
       'census_2011', 'projection_2015', 'projection_2021', 'projection_2031',
       'projection_2039'],
      dtype='object')


In [76]:
population_data.to_csv(
    '../data/processed/population_by_borough.csv',
    index=False,
    header=True,
    encoding='utf-8'
)